In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import logging
from ImmuneGWAS.helpers.getpaths import get_paths, get_sumstats_path
from ImmuneGWAS.helpers import dbsnp, ldlink
import ImmuneGWAS.resources.immune_GWAS as immune_GWAS
from ImmuneGWAS import config
from ImmuneGWAS.variant import Variant

from ImmuneGWAS.resources.eqtlgen import eqtlgen_trans_LDblock_query 
from ImmuneGWAS.resources.eqtlgen import eqtlgen_cis_LDblock_query 
from ImmuneGWAS.resources.tokyo_eqtl import tokyo_eqtl_LDblock_query
from ImmuneGWAS.resources.eqtl_cat import *
from IPython.display import display_html 
from IPython.display import display,HTML

from scipy.cluster import hierarchy

from ImmuneGWAS.helpers import get_gene_symbol  # convert ensembl to gene

In [19]:
ldpruned_hits = "/media/cbio3/projects/antton/Immune_cell_GWAS/data/hits_only_table_hg38_LDprunned.txt"

Designing the lookup:

1) Variant object calls trans_df lookup
2) Harmonize according to EA/OA, raise exceptions for multi-allelic SNPs
3) Split trans_df into up and down
4) Return row-scaled heatmap df where cell ordering is hard-coded and rows are resorted for clustering
5) Plotting function for heatmap df
6) Get flow phenotypes
7) Get LDTrait phenotypes
8) Summarize phenotpyes into tables with betas (remove duplicate phenotypes etc)
9) Get a single summary cis-eQTL table
10) Return summary report - 2 heatmaps for up and down, cis-eQTLs, phenotypes summarized

In [2]:
rsid = "rs1354034"
chrom = 3
pos = 56815721
EA = 'T'
OA = 'C'

x = Variant(rsid, chrom, pos, EA, OA)

In [ ]:
x.results.

In [62]:
def summary_trans(x: Variant):
    
    # retrieve trans-eQTLs
    eqtlgen_trans_LDblock_query(x)
    trans = x.results.eqtlgen_trans()
    trans.Zscore = trans.Zscore.astype(float)
    
    
    # check EA for sign of beta
    
    if set(trans.AssessedAllele == x.EA) == {True}:
        trans.Zscore = trans.Zscore.astype(float) * -1
    
    # split into up and down genes
    up = trans[trans.Zscore>0]
    down = trans[trans.Zscore<0]
    up = up.Gene
    down = down.Gene
    
    
    #get ge matrix from ImmunexUT
    # Get ge matrix from ImmunexUT
    paths = get_paths(root=config.cbio_root)
    ge = pd.read_csv(paths['ge_tokyo'], sep='\t')
    ge = ge.set_index("Gene_id")
    ge = ge.drop("Gene_name", axis=1)
    ge = ge.drop_duplicates()
    
    # get indices for trans-eQTL matrix
    # note any missing genes will be dropped
    
    u = [i for i in up if i in ge.index]
    u = set(u)
    d = [i for i in down if i in ge.index]
    d = set(d)
    
    # filter gene expression matrix into up and down genes
    u = ge.loc[u]
    d = ge.loc[d]
    
    # rename ensembl IDs to gene names
    u.index = u.index.map(mapper)
    d.index = d.index.map(mapper)

    # hard-code order of cell types according to gene expression similarity on whole GE matrix
    
    ordered_cols = ['Plasmablast', 'LDG', 'Neu', 'pDC', 'mDC', 'CL_Mono', 'Int_Mono',
           'CD16p_Mono', 'NC_Mono', 'Naive_B', 'USM_B', 'DN_B', 'SM_B', 'NK',
           'Mem_CD8', 'EM_CD8', 'TEMRA_CD8', 'Naive_CD8', 'Fr_I_nTreg',
           'Naive_CD4', 'Fr_III_T', 'Fr_II_eTreg', 'CM_CD8', 'Th2', 'Tfh', 'Th17',
           'Mem_CD4', 'Th1']
    
    # normalize up and down genes

    def standard_scale(x):
        return (x-x.min())/x.max()

    u = u.apply(standard_scale, axis=1)
    d = d.apply(standard_scale, axis=1)

    # reorder columns
    
    u = u.loc[:, ordered_cols]
    d = d.loc[:, ordered_cols]
    
    # reoder rows by dendrogram
    if len(u)>1:
        Z = hierarchy.linkage(u, optimal_ordering=True, method='ward')
        dendro = hierarchy.dendrogram(Z, no_plot=True)
    
        u = u.iloc[dendro['leaves'],:]
    
    if len(d)>1:
        Z = hierarchy.linkage(d, optimal_ordering=True, method='ward')
        dendro = hierarchy.dendrogram(Z, no_plot=True)
        d = d.iloc[dendro['leaves'],:]
    
    # collect all my cis-eqtls

    eqtlgen_cis_LDblock_query(x)
    tokyo_eqtl_LDblock_query(x)
    eqtl_catalogue_LDblock_query_type_restricted_multitype(x)

    # now all of these are in the Results object

    gwas = ldlink.ldtrait(x)

    # first try to filter by the same SNP as in Variant object, otherwise filter down to one rsid
    
    tokyo = x.results.tokyo_eqtl()[x.results.tokyo_eqtl()['Variant_ID']==x.rsid]
    
    if len(tokyo)>0:
        tokyo = tokyo[['Gene_name','Variant_ID', 'Backward_slope', 'Backward_P', 'cell_type']]
        tokyo = tokyo.sort_values(by='Gene_name')
    
    elif len(tokyo)==0:
        tokyo = x.results.tokyo_eqtl()
        rsid = tokyo['Variant ID'][0]
        

    
    cat = x.results.eqtl_cat()
    if len(cat)>0:
        cat = cat[cat.position==str(x.pos)]
        cat = cat[['gene_symbol', 'z', 'study', 'cell_type']]
    
    
    gwas_traits = set(x.results.ldtrait()['GWAS Trait'].tolist())
    
    
    
    cisgen = x.results.eqtlgen_cis()
    cisgen = cisgen[cisgen.SNP==x.rsid]
    if len(cisgen)>0:
        cisgen = cisgen[['SNP', 'Zscore', 'Pvalue', 'GeneSymbol']].sort_values(by='GeneSymbol')
    elif len(csigen)==0:
        cisgen = x.results.eqtlgen_cis()
        rsid = cisgen.SNP[0]
        cisgen = cisgen[cisgen.SNP==rsid]
        cisgen = cisgen[['SNP', 'Zscore', 'Pvalue', 'GeneSymbol']].sort_values(by='GeneSymbol')
        
    # get the cis-eqtl genes to plot a heatmap of the cis-eQTL genes
    
    if len(x.results.eqtlgen_cis())>0:
        a = x.results.eqtlgen_cis_df['Gene'].tolist()
    else:
        a = []

    if len(x.results.tokyo_eqtl())>0:
        b = x.results.tokyo_eqtl_df['Gene_id'].apply(lambda x:x.split(".")[0]).tolist()
    else:
        b = []
    
    if len(x.results.eqtl_cat())>0:
        c = x.results.eqtl_cat_df['molecular_trait_id'].tolist()
    else:
        c = []
    
    cis_genes = a + b + c
    cis_genes = [i for i in cis_genes if i in ge.index]
    cis_genes = list(set(cis_genes))
    cis_genes = ge.loc[cis_genes]
    cis_genes = cis_genes.apply(standard_scale, axis=1)
    cis_genes = cis_genes.loc[:, ordered_cols]
    cis_genes.index = cis_genes.index.map(get_gene_symbol)
    

    display(tokyo)
    display(cat)
    display(cisgen)
    print("GWAS Catalog\n")
    display(gwas_traits)
    print("Flow phenos\n")
    display(x.get_gwas_phenotypes())
    
    
    sns.heatmap(cis_genes, cbar=False, cmap='inferno', square=True)
    
    h = max([len(u), len(d)])
    h = int(h/10)
    if h<1:
        h=1
    
    if len(d)>0 and len(u)>0:
        print("both up and down trans")
        fig, ax= plt.subplots(1,2, figsize=(20,h))

        sns.heatmap(u, ax=ax[0], cbar=False, cmap='inferno')
        sns.heatmap(d, ax=ax[1], cbar=False, cmap='inferno')
    
        ax[0].set_title("UP Trans genes")
        ax[1].set_title("DOWN Trans genes")
    
    elif len(d)==0 and len(u)>0:
  
        fig, ax = plt.subplots(figsize=(10,h))
        sns.heatmap(u, ax=ax, cbar=False, cmap='inferno')
        ax.set_title("UP Trans genes")
    
    elif len(d)>0 and len(u)==0:

        fig, ax = plt.subplots(figsize=(10,h))
        sns.heatmap(d, ax=ax, cbar=False, cmap='inferno')
        ax.set_title("DOWN Trans genes")
                               
    elif len(d)==0 and len(u)==0:
        print("no significant trans-eQTL detected")
        

In [25]:
rsid = 'rs8073060'
EA = 'A'
OA = 'T'
pos = 35548243
chrom = 17

x = Variant(rsid, chrom, pos, EA, OA)


In [63]:
paths = get_paths(root=config.cbio_root)
ge = pd.read_csv(paths['ge_tokyo'], sep='\t')
ge = pd.read_csv(paths['ge_tokyo'], sep='\t')
ge = ge.set_index("Gene_id")
ge = ge.drop("Gene_name", axis=1)
ge = ge.drop_duplicates()

a = x.results.eqtlgen_cis_df['Gene'].tolist()
b = x.results.tokyo_eqtl_df['Gene_id'].apply(lambda x:x.split(".")[0]).tolist()
c = []

cis_genes = a + b + c
cis_genes = [i for i in cis_genes if i in ge.index]
cis_genes = list(set(cis_genes))

In [67]:
cis_genes

['ENSG00000172716',
 'ENSG00000255987',
 'ENSG00000154760',
 'ENSG00000236320',
 'ENSG00000205045',
 'ENSG00000005156',
 'ENSG00000267369',
 'ENSG00000108733',
 'ENSG00000267074',
 'ENSG00000267364',
 'ENSG00000270806']

In [69]:
for i in cis_genes:
    print(i)
    try:
        print(get_gene_symbol(i))
    except:
        print(i + " is missing!")

ENSG00000172716
SLFN11
ENSG00000255987
TOMM20P2
ENSG00000154760
SLFN13
ENSG00000236320
SLFN14
ENSG00000205045
SLFN12L
ENSG00000005156
LIG3
ENSG00000267369
TAF5LP1
ENSG00000108733
PEX12
ENSG00000267074
ENSG00000267074 is missing!
ENSG00000267364
ENSG00000267364 is missing!
ENSG00000270806
C17orf50


In [ ]:
eqtl_catalogue_LDblock_query_type_restricted_multitype(x)
